<a href="https://colab.research.google.com/github/how-to-train-your-model/data-science-portfolio/blob/main/MSDS_451_A7_Pant_Applying_CPCV_on_a_backtest_of_a_trading_strategy_on_SPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 7
  - In last week’s coding assignment, we observed that the Sharpe ratio of a backtest can be regarded as just one sample in a distribution of possible Sharpe ratios. For a backtest of a machine-learning-based strategy, we can generate even more samples by training different models based on different subsets of data. This is the idea behind Combinatorial Purged Cross Validation. We will apply that, again, to our trading strategy based on the SPX index monthly returns prediction.



# Libraries and prep

In [1]:
!pip install sweetviz
!pip install pyfolio
# !pip install mlfinlab --no-cache-dir
# !pip install mlfinlab



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 60.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 KB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 KB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.1 MB/s eta 0:00:00
  Created wheel for pyfolio: filename=pyfolio-0.9.2-py3-none-any.whl size=88681 sha256=1fbd5b68ee04efc1e3bde1d82b8050a89e440cf0f7ae850b943a67f6447a9156
  Stored in directory: /root/.cache/pip/wheels/42/16/92/9c9d49efccc497040268960f0825101cc9b1c6e41cdc5a409e
  Created wheel for empyrical: filename=empyrical-0.5.5-py3-none-any.whl size=39778 sha256=eb7de717500cefe2e50e15d8929e4967796f231c50d1bc9a3b

In [2]:
# load libraries
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score, cross_validate

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import sweetviz as sv
from sklearn.impute import SimpleImputer
from datetime import datetime
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import log_loss,accuracy_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestRegressor
import warnings
import pickle
import pyfolio as pf


/usr/local/lib/python3.8/dist-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [3]:
# Allows for muliple output in one cell window
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Allows for viewing the complete data in a single window 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

warnings.simplefilter('ignore')


In [4]:
from google.colab import drive
drive.mount('/content/drive')
!ls -l "/content/drive/My Drive/Colab_Notebooks/MSDS-451/W7"

Mounted at /content/drive
total 9867
-rw------- 1 root root 4721852 Mar  5 21:01  ctreeTemp.sav
-rw------- 1 root root    4849 Feb 27 00:27 'getCPCVtestset (1).ipynb'
-rw------- 1 root root  292271 Mar  5 23:58 'MSDS-451_A7_Pant_Applying CPCV on a backtest of a trading strategy on SPY.docx'
-rw------- 1 root root   33160 Mar  6 00:01 'MSDS-451_A7_Pant_Applying CPCV on a backtest of a trading strategy on SPY.ipynb'
-rw------- 1 root root  225741 Mar  5 23:35 'MSDS 451 Assignment_06_Chan-1.ipynb'
-rw------- 1 root root 4541224 Mar  5 20:36  random_search.sav
-rw------- 1 root root  244386 Mar  4 18:38  S_P_mnthly_returns_input-3.csv
-rw------- 1 root root   13711 Mar  4 18:38  S_P_mnthly_returns_output-3.csv
-rw------- 1 root root   24148 Mar  4 18:50  SPY.csv


In [5]:
w7_files = "/content/drive/My Drive/Colab_Notebooks/MSDS-451/W7"

#Q1
Suppose you have a data set of 10 bars, which include a test set of 2 bars. Denote those bars 1,2, 3...10. Produce 3 examples of distinct backtest paths, identify the train sets and test sets used in each path. (3 pt)

## Response:

With test set of 2 bars out of 10 bars; we can have total of 45 possible combinations of train and test sets that can be used for backtesting this data set.

10C2 = 10! / (2! * (10-2)!)

**=45**

# Q2

For our SPX data set, suppose we pick 5 groups of training sets, and with test sets of size 2, create the set of backtest paths using get CPCVtestset function provided and compute their Sharpe ratios. What are the mean and standard deviation of this set of Sharpe ratios? How do they compare with the Sharpe ratios computed on the train and test sets in the Parts 1 and 3 of last week’s assignment? (7 pt)

In [6]:
X_data = pd.read_csv(w7_files+"/S_P_mnthly_returns_input-3.csv")
y_data = pd.read_csv(w7_files+"/S_P_mnthly_returns_output-3.csv")

X_data.head(2)
y_data.head(2)
X_data.dtypes
#y_data.dtypes

,yyyymm,ntis,infl,ltr,corpr,svar,d/p,d/y,e/p,d/e,tms,dfy,dfr,b/m_FD,tbl_FD,lty_FD
0,1945-12-01,0.025525,0.005525,0.0194,0.0133,0.001538,-3.269684,-3.337063,-2.894991,-0.374693,0.0161,0.0049,-0.0061,-0.216009,-2.99635,-1.507835
1,1946-01-01,0.027969,0.000000,0.0025,0.0128,0.002645,-3.327012,-3.255014,-2.983423,-0.343589,0.0161,0.0047,0.0103,-0.264780,-2.99635,-1.486065


,yyyymm,ret
0,1945-12-01,1.0
1,1946-01-01,-1.0


yyyymm     object
ntis      float64
infl      float64
ltr       float64
corpr     float64
svar      float64
d/p       float64
d/y       float64
e/p       float64
d/e       float64
tms       float64
dfy       float64
dfr       float64
b/m_FD    float64
tbl_FD    float64
lty_FD    float64
dtype: object

In [7]:

# setting the date column as index for the dataset.
# X_data = X_data.set_index('yyyymm')
# y_data = y_data.set_index('yyyymm')
X_data['yyyymm'] = pd.to_datetime(X_data['yyyymm'])
y_data['yyyymm'] = pd.to_datetime(y_data['yyyymm'])

X_data.set_index('yyyymm', inplace = True)
y_data.set_index('yyyymm', inplace = True)

In [8]:
X_data.head()

,ntis,infl,ltr,corpr,svar,d/p,d/y,e/p,d/e,tms,dfy,dfr,b/m_FD,tbl_FD,lty_FD
yyyymm,,,,,,,,,,,,,,,
1945-12-01,0.025525,0.005525,0.0194,0.0133,0.001538,-3.269684,-3.337063,-2.894991,-0.374693,0.0161,0.0049,-0.0061,-0.216009,-2.99635,-1.507835
1946-01-01,0.027969,0.000000,0.0025,0.0128,0.002645,-3.327012,-3.255014,-2.983423,-0.343589,0.0161,0.0047,0.0103,-0.264780,-2.99635,-1.486065
1946-02-01,0.031091,-0.005495,0.0032,0.0034,0.005942,-3.245065,-3.290322,-2.932931,-0.312134,0.0160,0.0047,0.0002,-0.167204,-2.99635,-1.481915
1946-03-01,0.031188,0.011050,0.0010,0.0034,0.001497,-3.280469,-3.317389,-3.000167,-0.280302,0.0160,0.0047,0.0024,-0.190201,-2.99635,-1.474585
1946-04-01,0.031307,0.005464,-0.0135,-0.0043,0.001078,-3.317389,-3.339531,-3.059560,-0.257829,0.0169,0.0050,0.0092,-0.222940,-2.99635,-1.425691


## EDA

In [9]:
dfq1 = X_data.copy()
dfq1.head(2)
df_EDA = dfq1.copy()

,ntis,infl,ltr,corpr,svar,d/p,d/y,e/p,d/e,tms,dfy,dfr,b/m_FD,tbl_FD,lty_FD
yyyymm,,,,,,,,,,,,,,,
1945-12-01,0.025525,0.005525,0.0194,0.0133,0.001538,-3.269684,-3.337063,-2.894991,-0.374693,0.0161,0.0049,-0.0061,-0.216009,-2.99635,-1.507835
1946-01-01,0.027969,0.000000,0.0025,0.0128,0.002645,-3.327012,-3.255014,-2.983423,-0.343589,0.0161,0.0047,0.0103,-0.264780,-2.99635,-1.486065


In [10]:
print("\n\n Duplicates",df_EDA.duplicated().sum())
#my_report = sv.analyze(df_EDA)
#my_report.show_html()



 Duplicates 0


In [11]:
print("\n\n Missing Values before dropping/Imputing \n",df_EDA.isna().sum())



 Missing Values before dropping/Imputing 
 ntis      0
infl      0
ltr       0
corpr     0
svar      0
d/p       0
d/y       0
e/p       0
d/e       0
tms       0
dfy       0
dfr       0
b/m_FD    0
tbl_FD    0
lty_FD    0
dtype: int64


## Preparing training data

In [12]:
y_data.index

DatetimeIndex(['1945-12-01', '1946-01-01', '1946-02-01', '1946-03-01', '1946-04-01', '1946-05-01', '1946-06-01', '1946-07-01', '1946-08-01', '1946-09-01',
               ...
               '2019-02-01', '2019-03-01', '2019-04-01', '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01', '2019-09-01', '2019-10-01', '2019-11-01'], dtype='datetime64[ns]', name='yyyymm', length=888, freq=None)

In [13]:
X_data = X_data[X_data.index.isin(y_data.index)]

In [14]:
X_data.shape

(888, 15)

In [15]:
start_date = '2012-06-01'
end_date = '2019-10-01'

In [16]:
X_train = X_data[:start_date]
X_test = X_data[start_date:]
y_train = y_data[:start_date]
y_test = y_data[start_date:]



In [17]:
X_train.shape
X_test.shape
y_train.shape
y_test.shape
#X_test = X_test.drop('2019-11-01')
X_test.tail()


(799, 15)

(90, 15)

(799, 1)

(90, 1)

,ntis,infl,ltr,corpr,svar,d/p,d/y,e/p,d/e,tms,dfy,dfr,b/m_FD,tbl_FD,lty_FD
yyyymm,,,,,,,,,,,,,,,
2019-07-01,-0.012703,0.001671,0.0024,0.0084,0.000594,-3.966309,-3.948051,-3.098391,-0.867918,-0.0004,0.0099,0.0060,-0.554641,-2.113560,-1.525435
2019-08-01,-0.010244,-0.000051,0.0797,0.0738,0.004318,-3.941330,-3.958365,-3.086025,-0.855305,-0.0032,0.0089,-0.0059,-0.531192,-2.175405,-1.735233
2019-09-01,-0.010959,0.000783,-0.0192,-0.0190,0.000605,-3.951689,-3.971915,-3.108987,-0.842702,-0.0019,0.0088,0.0002,-0.553503,-2.185571,-1.606937
2019-10-01,-0.013267,0.002286,-0.0052,0.0006,0.001510,-3.965984,-3.999464,-3.112869,-0.853115,0.0006,0.0091,0.0058,-0.552792,-2.304826,-1.576943
2019-11-01,-0.007907,-0.000536,-0.0059,0.0014,0.000306,-3.993568,-4.021757,-3.130267,-0.863301,0.0027,0.0088,0.0073,-0.585980,-2.337228,-1.503722


In [18]:
#y_test = y_test.drop('2019-11-01')


In [19]:
# From Scott: Create t1 (Lopez de Prado nomenclature) dataframes of times
Ytrain_time = pd.Series(y_train.index,index = X_train.index)
Ytest_time = pd.Series(y_test.index, index = X_test.index)

In [20]:
Ytrain_time.head(2)
Ytest_time.head(2)

yyyymm
1945-12-01   1945-12-01
1946-01-01   1946-01-01
Name: yyyymm, dtype: datetime64[ns]

yyyymm
2012-06-01   2012-06-01
2012-07-01   2012-07-01
Name: yyyymm, dtype: datetime64[ns]

## Backtesting through Cross-Validation 
--- The Combinatorial Cross-Validation Method: CPCV

In [21]:
from itertools import combinations

def getCPCVtestset(T,N,k):
    '''
      (isTest, paths)  = getCPCVtestset(T, N, k)
      Get train and test sets in LdP's CPCV. 
    
      T = length of data
      N=number of groups
      k = number of groups in test set. 
    
      isTest is a T x C(N, k) array, where each column is a logical array indicating
      which observation is in test set (the false ones are in train set) for a particular split.
      paths is a T x k/N C(N, k) array, where each column represents one
      backtest path, and each row denotes the column of isTest to be used for
      backtest.
    '''
    groupNum=np.full((T,1),np.nan)
    groupSize=np.floor(T/N).astype(np.int)
    for g in range(N-1):
        groupNum[range(g*groupSize,(g+1)*groupSize)]=g
    groupNum[range((g+1)*groupSize,groupNum.shape[0])]=N-1
    groupNum=groupNum.ravel().astype(int)

    # This block is my rendition of the following short Matlab line:
    # testGroups=combnk(1:N, k);
    # All of the extra effort is to convert the iterator combinations(range(1,N+1),k)
    # into the realized array of groups.
    testGroups=np.array([])
    for comb in combinations(range(N),k):
        testGroups=np.append(testGroups,np.array(comb))
    testGroups=testGroups.astype(int).reshape(np.floor(testGroups.shape[0]/k).astype(int),k)

    isTestGroup=np.full((N,testGroups.shape[0]),False) #% isTestGroup(g, s)=true denotes group g in split s is a test set.
    isTest=np.full((T,testGroups.shape[0]),False)
    for s in range(isTest.shape[1]):
        for g in range(testGroups.shape[1]):
            isTestGroup[testGroups[s,g],s]=True 
            isTest[groupNum==testGroups[s,g],s]=True

    mypaths=np.full((np.unique(groupNum).shape[0],np.floor(k*testGroups.shape[0]/N).astype(int)),np.nan)    

    for p in range(mypaths.shape[1]):
        for g in range(mypaths.shape[0]):
            s=isTestGroup[g,:].argmax()
            mypaths[g,p]=s
            isTestGroup[g,s]=False
    mypaths=mypaths.astype(int)

    paths=np.full((T,mypaths.shape[1]),np.nan)
    for p in range(mypaths.shape[1]):
        for g in range(mypaths.shape[0]):
            paths[groupNum==g,p]=mypaths[g,p]
    paths=paths.astype(int)

    return (isTest,paths)

### 2a
For our SPX data set, suppose we pick 5 groups of training sets, and with test sets of size 2, create the set of backtest paths using get CPCVtestset function provided and compute their Sharpe ratios. 

In [22]:
# suppose we pick 5 groups of training sets with test sets of size 2
numGroups=5 
numGroupsTest=2 # If =1, always gives 1 path: CPCV -> CV
trainFrac=1 # Fraction of train set to be added to valid set in CPCV

from itertools import combinations
numCombs=0
for comb in combinations(range(numGroups),numGroupsTest):
    numCombs+=1

isTest,paths=getCPCVtestset(X_test.shape[0],numGroups,numGroupsTest)
print('isTest:',isTest.shape,'paths:',paths.shape)
pred=np.full(isTest.shape,np.nan) # OOS prediction on each row. Each column represents a different split of data between train and test.
prob=np.full(isTest.shape,np.nan)

print('CPCV has',paths.shape[1], 'paths')

isTest: (90, 10) paths: (90, 4)
CPCV has 4 paths


In [23]:
isTest.shape
isTest[1]

(90, 10)

array([ True,  True,  True,  True, False, False, False, False, False,
       False])

In [24]:
pred[:,0]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [25]:
prob[:,0]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [26]:
isTest[:,0]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [27]:
isTest.shape
X_train.shape
y_train.shape
type(isTest)

(90, 10)

(799, 15)

(799, 1)

numpy.ndarray

In [28]:
pred[:,0]
pred.shape

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

(90, 10)

In [29]:
X_test.head(2)
y_test.head(2)
X_test.shape
y_test.shape
y_data.shape

,ntis,infl,ltr,corpr,svar,d/p,d/y,e/p,d/e,tms,dfy,dfr,b/m_FD,tbl_FD,lty_FD
yyyymm,,,,,,,,,,,,,,,
2012-06-01,-0.019820,-0.001466,-0.0136,0.0064,0.003437,-3.873259,-3.885778,-2.740400,-1.132859,0.0224,0.0138,0.0200,-0.392624,-3.580049,-1.437708
2012-07-01,-0.021091,-0.001630,0.0247,0.0612,0.001553,-3.870940,-3.890511,-2.758317,-1.112623,0.0206,0.0147,0.0365,-0.396861,-3.521716,-1.503595


,ret
yyyymm,
2012-06-01,1.0
2012-07-01,1.0


(90, 15)

(90, 1)

(888, 1)

## Train a model using CPCV

In [30]:
X_train.shape
y_train.shape
X_test.shape
y_test.shape

(799, 15)

(799, 1)

(90, 15)

(90, 1)

In [31]:
from sklearn.model_selection import cross_val_score, KFold

# Define the random forest model
rf = RandomForestClassifier(n_estimators=100, max_depth=5)

# Define the cross-validation strategy (here, we use KFold with 5 folds)
cv = KFold(n_splits=5)

# Train and test the model using cross-validation
scores = cross_val_score(rf, X_train, y_train, cv=cv, scoring='accuracy')

# Print the average accuracy score
print(f"Average accuracy: {scores.mean()}")

Average accuracy: 0.5631761006289308


## SPY_DATA

In [32]:
# From A6

SPY_data = pd.read_csv(w7_files+ "/SPY.csv")
SPY_data.head(2)

,Date,Open,High,Low,Close,Adj Close,Volume
0,1993-02-01,43.96875,45.12500,42.81250,44.40625,26.160336,5417600
1,1993-03-01,44.56250,45.84375,44.21875,45.18750,26.620569,3019200


In [33]:
SPY_data['Date']= pd.to_datetime(SPY_data['Date'])


In [34]:
# From A6 solution code snippet:
import datetime

SPY_data.head(2)
start_date = '2012-06-01'
end_date = '2019-10-01'

SPY_data_fil = SPY_data.loc[( SPY_data['Date'] >= 
                             datetime.datetime(2012, 6, 1) )]
SPY_data_fil = SPY_data_fil.loc[( SPY_data['Date'] <= 
                                 datetime.datetime(2019, 10, 1) )]

SPY_data_fil.shape
SPY_data_fil.tail(5)



,Date,Open,High,Low,Close,Adj Close,Volume
0,1993-02-01,43.96875,45.12500,42.81250,44.40625,26.160336,5417600
1,1993-03-01,44.56250,45.84375,44.21875,45.18750,26.620569,3019200


(89, 7)

,Date,Open,High,Low,Close,Adj Close,Volume
316,2019-06-01,275.309998,296.309998,273.089996,293.000000,282.601105,1340435600
317,2019-07-01,296.679993,302.230011,294.329987,297.429993,288.269165,1110102300
318,2019-08-01,297.600006,300.869995,281.720001,292.450012,283.442535,2034004800
319,2019-09-01,290.570007,302.630005,289.269989,296.769989,287.629486,1303830000
320,2019-10-01,297.739990,304.549988,284.820007,303.329987,295.345062,1386748300


In [35]:
perf_func = pf.timeseries.perf_stats

def get_monthly_returns(monthly_returns):
    """
    This changes returns into monthly returns that will work using pyfolio. Its not perfect...
    """
    
    cum_rets = ((monthly_returns + 1).cumprod())

    # Downsample to monthly
    monthly_rets = cum_rets.resample('M').last()

    # Forward fill, Percent Change, Drop NaN
    monthly_rets = monthly_rets.ffill().pct_change().dropna()
    
    return monthly_rets

In [36]:
SPY_data_SR = SPY_data_fil.copy(deep = True)
SPY_data_SR.set_index('Date' , inplace=True)


## Sharpe Calculation

In [37]:
start_date = '2012-06-01'
end_date = '2019-10-01'

In [38]:
X_train = X_data[:start_date]
X_test = X_data[start_date:]
y_train = y_data[:start_date]
y_test = y_data[start_date:]



In [39]:
X_data.shape
y_data.shape
X_train.shape
X_test.shape
y_train.shape
y_test.shape
#X_test = X_test.drop('2019-11-01')
X_test.tail()


(888, 15)

(888, 1)

(799, 15)

(90, 15)

(799, 1)

(90, 1)

,ntis,infl,ltr,corpr,svar,d/p,d/y,e/p,d/e,tms,dfy,dfr,b/m_FD,tbl_FD,lty_FD
yyyymm,,,,,,,,,,,,,,,
2019-07-01,-0.012703,0.001671,0.0024,0.0084,0.000594,-3.966309,-3.948051,-3.098391,-0.867918,-0.0004,0.0099,0.0060,-0.554641,-2.113560,-1.525435
2019-08-01,-0.010244,-0.000051,0.0797,0.0738,0.004318,-3.941330,-3.958365,-3.086025,-0.855305,-0.0032,0.0089,-0.0059,-0.531192,-2.175405,-1.735233
2019-09-01,-0.010959,0.000783,-0.0192,-0.0190,0.000605,-3.951689,-3.971915,-3.108987,-0.842702,-0.0019,0.0088,0.0002,-0.553503,-2.185571,-1.606937
2019-10-01,-0.013267,0.002286,-0.0052,0.0006,0.001510,-3.965984,-3.999464,-3.112869,-0.853115,0.0006,0.0091,0.0058,-0.552792,-2.304826,-1.576943
2019-11-01,-0.007907,-0.000536,-0.0059,0.0014,0.000306,-3.993568,-4.021757,-3.130267,-0.863301,0.0027,0.0088,0.0073,-0.585980,-2.337228,-1.503722


In [40]:
X_test3 = X_test.copy(deep = True)
#X_test3.dropna(inplace=True)
#X_test3.set_index('yyyymm' , inplace=True)
X_test3.head()


,ntis,infl,ltr,corpr,svar,d/p,d/y,e/p,d/e,tms,dfy,dfr,b/m_FD,tbl_FD,lty_FD
yyyymm,,,,,,,,,,,,,,,
2012-06-01,-0.019820,-0.001466,-0.0136,0.0064,0.003437,-3.873259,-3.885778,-2.740400,-1.132859,0.0224,0.0138,0.0200,-0.392624,-3.580049,-1.437708
2012-07-01,-0.021091,-0.001630,0.0247,0.0612,0.001553,-3.870940,-3.890511,-2.758317,-1.112623,0.0206,0.0147,0.0365,-0.396861,-3.521716,-1.503595
2012-08-01,-0.020536,0.005565,-0.0068,-0.0093,0.000727,-3.875890,-3.899837,-2.783315,-1.092575,0.0213,0.0143,-0.0025,-0.398770,-3.585436,-1.440500
2012-09-01,-0.018153,0.004462,-0.0146,-0.0126,0.001045,-3.885427,-3.865439,-2.812719,-1.072708,0.0224,0.0135,0.0020,-0.420618,-3.526902,-1.384686
2012-10-01,-0.016443,-0.000389,-0.0014,0.0206,0.000926,-3.846946,-3.849788,-2.792693,-1.054253,0.0228,0.0111,0.0220,-0.385622,-3.676852,-1.381841


In [41]:
numGroups=5
numGroupsTest=2 # If =1, always gives 1 path: CPCV -> CV
trainFrac=1 # Fraction of train set to be added to valid set in CPCV

from itertools import combinations
numCombs=0
for comb in combinations(range(numGroups),numGroupsTest):
    numCombs+=1

isTest,paths=getCPCVtestset(X_test3.shape[0],numGroups,numGroupsTest)
print('isTest:',isTest.shape,'paths:',paths.shape)
pred=np.full(isTest.shape,np.nan) # OOS prediction on each row. Each column represents a different split of data between train and test.
prob=np.full(isTest.shape,np.nan)

print('CPCV has',paths.shape[1], 'paths')

isTest: (90, 10) paths: (90, 4)
CPCV has 4 paths


In [42]:
paths.shape[1]
isTest.shape
isTest[:,2]

4

(90, 10)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [43]:
SPY_data_SR.head(10)
SPY_data_SR.shape
X_test3.head(10)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-06-01,129.410004,136.270004,127.139999,136.100006,114.001549,3644886200
2012-07-01,136.479996,139.339996,132.600006,137.710007,115.947792,2864743300
2012-08-01,138.699997,143.089996,135.580002,141.160004,118.852539,2413590900
2012-09-01,141.039993,148.110001,140.130005,143.970001,121.218559,2391233500
2012-10-01,144.520004,147.160004,140.389999,141.350006,119.647865,2719915500
2012-11-01,141.649994,143.720001,134.699997,142.149994,120.325035,3032769100
2012-12-01,142.800003,145.580002,139.539993,142.410004,120.545143,2889875900
2013-01-01,145.110001,150.940002,144.729996,149.699997,127.614578,2587140200
2013-02-01,150.649994,153.279999,148.729996,151.610001,129.242783,2581459300


(89, 6)

,ntis,infl,ltr,corpr,svar,d/p,d/y,e/p,d/e,tms,dfy,dfr,b/m_FD,tbl_FD,lty_FD
yyyymm,,,,,,,,,,,,,,,
2012-06-01,-0.019820,-0.001466,-0.0136,0.0064,0.003437,-3.873259,-3.885778,-2.740400,-1.132859,0.0224,0.0138,0.0200,-0.392624,-3.580049,-1.437708
2012-07-01,-0.021091,-0.001630,0.0247,0.0612,0.001553,-3.870940,-3.890511,-2.758317,-1.112623,0.0206,0.0147,0.0365,-0.396861,-3.521716,-1.503595
2012-08-01,-0.020536,0.005565,-0.0068,-0.0093,0.000727,-3.875890,-3.899837,-2.783315,-1.092575,0.0213,0.0143,-0.0025,-0.398770,-3.585436,-1.440500
2012-09-01,-0.018153,0.004462,-0.0146,-0.0126,0.001045,-3.885427,-3.865439,-2.812719,-1.072708,0.0224,0.0135,0.0020,-0.420618,-3.526902,-1.384686
2012-10-01,-0.016443,-0.000389,-0.0014,0.0206,0.000926,-3.846946,-3.849788,-2.792693,-1.054253,0.0228,0.0111,0.0220,-0.385622,-3.676852,-1.381841
2012-11-01,-0.012811,-0.004738,0.0144,-0.0092,0.001903,-3.831631,-3.838674,-2.795497,-1.036134,0.0219,0.0101,-0.0236,-0.384276,-3.800781,-1.429669
2012-12-01,-0.012258,-0.002693,-0.0202,-0.0062,0.001036,-3.820840,-3.870038,-2.802502,-1.018338,0.0239,0.0098,0.0140,-0.392551,-4.043244,-1.342220
2013-01-01,-0.008679,0.002957,-0.0332,-0.0313,0.000907,-3.860853,-3.871853,-2.847125,-1.013728,0.0284,0.0093,0.0019,-0.447843,-3.985734,-1.193066
2013-02-01,-0.009526,0.008190,0.0114,0.0093,0.001243,-3.862751,-3.898106,-2.853571,-1.009180,0.0275,0.0095,-0.0021,-0.445050,-3.606380,-1.269018


In [49]:
Sharpe_Ratio=np.full((paths.shape[1]),np.nan)

for i in range(paths.shape[1]):
  X_train11 = X_test3[~isTest[:,i]]
  X_test11 = X_test3[isTest[:,i]]
  y_train11 = y_test[~isTest[:,i]]
  y_test11 = y_test[isTest[:,i]]
  X_train11.shape
  y_train11.shape

  #X_train11.head(2)
  #y_train11.head(2)

  # clf = RandomForestClassifier(n_estimators = 100 ,
  #                              n_jobs = 4, 
  #                              max_depth=5,
  #                              criterion = 'entropy',
  #                              random_state = 1)
  
  base_est = DecisionTreeClassifier(criterion='entropy', 
                                      max_features='auto', 
                                      class_weight='balanced', 
                                      min_weight_fraction_leaf=0.05)
  clf = BaggingClassifier(base_estimator=base_est, 
                          n_estimators=50, 
                          max_features=.99)
  


  # clf = RandomForestClassifier(n_estimators=100, max_depth=5,
  #                              random_state = 1)

  # clf_random = RandomizedSearchCV(estimator = ctree, 
  #                                 param_distributions = param_distributions,
  #                                 scoring = 'f1', 
  #                                 cv = cv_a7, n_jobs = jobs, 
  #                                 pre_dispatch = jobs,
  #                                 n_iter = iter,
  #                                 verbose=10, random_state = 1)
  clf.fit(X_train11, y_train11)
  pred = clf.predict(X_test11)

  X_test11['side'] = pred
  #joined_df1 = SPY_data_SR.join(X_test11)
  #joined_df1.shape
  sigs = X_test11[X_test11.side.shift(1)!=X_test11.side]
  sigs=sigs.iloc[1:,:]
  sigs['close'] = np.nan
  for j in sigs.index:
    sigs.loc[j, 'close'] = SPY_data_SR.loc[j,'Adj Close']

  #sigs.head()
  sigs['ret']=(sigs.close.shift(-1)-sigs.close)/sigs.close*sigs.side
  monthly_rets = get_monthly_returns(sigs['ret'])
  Sharpe_Ratio[i]=np.mean(monthly_rets)/np.std(monthly_rets)*np.sqrt(12)
  #pf.show_perf_stats(monthly_rets)


Sharpe_Ratio 

(54, 15)

(54, 1)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight='balanced',
                                                        criterion='entropy',
                                                        max_features='auto',
                                                        min_weight_fraction_leaf=0.05),
                  max_features=0.99, n_estimators=50)

(54, 15)

(54, 1)

...

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight='balanced',
                                                        criterion='entropy',
                                                        max_features='auto',
                                                        min_weight_fraction_leaf=0.05),
                  max_features=0.99, n_estimators=50)

(54, 15)

(54, 1)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight='balanced',
                                                        criterion='entropy',
                                                        max_features='auto',
                                                        min_weight_fraction_leaf=0.05),
                  max_features=0.99, n_estimators=50)

array([0.73330692, 0.62355747, 0.51532897, 0.49333411])

In [45]:
pd.Series(Sharpe_Ratio).describe()

count    3.000000
mean     0.858499
std      0.578424
min      0.367814
25%      0.539616
50%      0.711418
75%      1.103841
max      1.496265
dtype: float64

# With IMP features

In [46]:
np.random.seed(1)

X_test4 = X_test3[['ntis', 'd/p', 'ltr', 'corpr','b/m_FD']]
X_test4.head()

#X_test4 = X_test3[['ntis', 'tms', 'dfr', 'corpr','b/m_FD']]
y_test.head()

,ntis,d/p,ltr,corpr,b/m_FD
yyyymm,,,,,
2012-06-01,-0.019820,-3.873259,-0.0136,0.0064,-0.392624
2012-07-01,-0.021091,-3.870940,0.0247,0.0612,-0.396861
2012-08-01,-0.020536,-3.875890,-0.0068,-0.0093,-0.398770
2012-09-01,-0.018153,-3.885427,-0.0146,-0.0126,-0.420618
2012-10-01,-0.016443,-3.846946,-0.0014,0.0206,-0.385622


,ret
yyyymm,
2012-06-01,1.0
2012-07-01,1.0
2012-08-01,1.0
2012-09-01,-1.0
2012-10-01,1.0


In [47]:
Sharpe_Ratio=np.full((paths.shape[1]),np.nan)

for i in range(paths.shape[1]):
  X_train11 = X_test4[~isTest[:,i]]
  X_test11 = X_test4[isTest[:,i]]
  y_train11 = y_test[~isTest[:,i]]
  y_test11 = y_test[isTest[:,i]]
  X_train11.shape
  y_train11.shape

  #X_train11.head(2)
  #y_train11.head(2)

  # clf = RandomForestClassifier(n_estimators = 100 ,
  #                              n_jobs = 4, 
  #                              max_depth=5,
  #                              criterion = 'entropy')
  
  base_est = DecisionTreeClassifier(criterion='entropy', 
                                      max_features='auto', 
                                      class_weight='balanced', 
                                      min_weight_fraction_leaf=0.05)
  clf = BaggingClassifier(base_estimator=base_est, n_estimators=50, max_features=.99)
  
  clf.fit(X_train11, y_train11)
  pred = clf.predict(X_test11)

  X_test11['side'] = pred
  #joined_df1 = SPY_data_SR.join(X_test11)
  #joined_df1.shape
  sigs = X_test11[X_test11.side.shift(1)!=X_test11.side]
  sigs=sigs.iloc[1:,:]
  sigs['close'] = np.nan
  for j in sigs.index:
    sigs.loc[j, 'close'] = SPY_data_SR.loc[j,'Adj Close']

  #sigs.head()
  sigs['ret']=(sigs.close.shift(-1)-sigs.close)/sigs.close*sigs.side
  monthly_rets = get_monthly_returns(sigs['ret'])
  Sharpe_Ratio[i]=np.mean(monthly_rets)/np.std(monthly_rets)*np.sqrt(12)
  #pf.show_perf_stats(monthly_rets)


Sharpe_Ratio 

(54, 5)

(54, 1)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight='balanced',
                                                        criterion='entropy',
                                                        max_features='auto',
                                                        min_weight_fraction_leaf=0.05),
                  max_features=0.99, n_estimators=50)

(54, 5)

(54, 1)

...

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight='balanced',
                                                        criterion='entropy',
                                                        max_features='auto',
                                                        min_weight_fraction_leaf=0.05),
                  max_features=0.99, n_estimators=50)

(54, 5)

(54, 1)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight='balanced',
                                                        criterion='entropy',
                                                        max_features='auto',
                                                        min_weight_fraction_leaf=0.05),
                  max_features=0.99, n_estimators=50)

array([ 0.68151809, -0.21468898, -0.02544191,  0.35736059])

In [48]:
pd.Series(Sharpe_Ratio).describe()

count    4.000000
mean     0.199687
std      0.399755
min     -0.214689
25%     -0.072754
50%      0.165959
75%      0.438400
max      0.681518
dtype: float64